# Equação de reacção-difusão em 2D


Tal como o método de Crank-Nicolson (CN) se aplica na resolução da equação de sistemas de reação-difusão com 1 dimensão espacial,  o método das Direções Alternadas Implícitas (Alternating Direction Implicit, ADI) é usado para a resolução da equação de sistemas de reação-difusão com 2 (ou mais) dimensões espaciais.

O método ADI é descrito em muitas fontes (ver [1][2], p.ex.).

No que se segue assumimos um sistema de reação-difusão com condições fronteira (CF) de Neumann.

$\begin{eqnarray*}
\frac{\partial u}{\partial t} = D\Bigl(\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2}\Bigr)+f(u),\\
\left.\frac{\partial u}{\partial x}\right|_{x=0,L_x} =0, \\
\left.\frac{\partial u}{\partial y}\right|_{y=0,L_y} =0
\end{eqnarray*}
$

onde $u(x,y,t)$ é a variável (dependente) que descreve a concentração, $D$ é  o coeficiente de difusão de u, f é o termo de reacção, e $L_x$ e $L_y$ são as dimensões do nosso domínio nas direcções $x$ e $y$, respectivamente.

Como o ADI e o CN estão relacionados, vamos ver as propriedades mais elementares do ADI relativamente às do CN.

### Grelhas

<img src="ADIfig1.png" alt="drawing" width="350"/>

Tal como para o CN construímos uma grelha $(x,t)$, também para o ADI precisamos construir uma grelha espacial $(x,y)$ nas duas dimensões espaciais (ver Fig.1).

No caso mais simples construímos uma grelha regular (cartesiana) do seguinte modo:  

$  \begin{eqnarray*}
t_n = n\Delta t,\ \ \ \ \  n=0,1,\ldots,N-1, \\
x_j = j\Delta x,\ \ \ \ \ \  j=0,1,\ldots,J-1, \\
y_i = i\Delta y,\ \ \ \ \ \  i=0,1,\ldots,I-1, 
\end{eqnarray*}
$  

onde $N,J,I$ são o número de pontos da grelha nas direcções $t, x, y,$ respectivamente. Os passos de cada grelha $\Delta t, \Delta x, \Delta y$ são definidos por:

$$ \Delta t = \frac{T}{N-1}, \ \ \ \ \  \Delta x = \frac{L_x}{J-1}, \ \ \ \ \  \Delta y = \frac{L_y}{I-1},  $$ 
onde $T$ é o tempo máximo que pretendemos.

Usamos doravante a notação $U(j\Delta x,i\Delta y,n\Delta t) \approx u(j\Delta x,i\Delta y,n\Delta t)$, em que $U$ é a aproximação numérica à função analítica desconhecida $u$, e usamos a notação abreviada $U(j\Delta x,i\Delta y,n\Delta t) = U^n_{j,i}$. Do mesmo modo usamos para o ponto $(j\Delta x, i\Delta y, n\Delta t)=(j, i, n).$

Podemos aplicar o mesmo stencil que usamos para CN a cada uma das derivadas espaciais (média entre tempos $n$ e $n+1$):

$
 \begin{eqnarray*}
\frac{U^{n+1}_{j,i} - U^n_{j,i}}{\Delta t} = & \frac{D}{2\Delta x^2}\left( U^n_{j+1,i} -2U^n_{j,i} + U^n_{j-1,i} + U^{n+1}_{j+1,i} -2U^{n+1}_{j,i} +U^{n+1}_{j-1,i}\right)\\
+ & \frac{D}{2\Delta y^2}\left( U^n_{j,i+1} -2U^n_{j,i} + U^n_{j,i-1} + U^{n+1}_{j,i+1} -2U^{n+1}_{j,i} +U^{n+1}_{j,i-1}\right) + f(U^n_{j,i})
\end{eqnarray*}
$

Sejam $\sigma_x \equiv \frac{D\Delta t}{2\Delta x^2}$ e $\sigma_y \equiv \frac{D\Delta t}{2\Delta y^2}$. Além disso, para reordenar as nossas equações para um sistema linear, precisamos de definir um novo índice, que combina $i$ e $j$ de acordo com a nossa escolha de percorrer a grelha bidimensional espacial. Escolhendo a ordem "column-major" (índice das linhas varia mais rapidamente, isto é, concatenamos cada coluna no fim da anterior) temos:

$$k = j + iJ$$

(_Nota_: A expressão para $k$ depende do primeiro valor do índice, e neste caso corresponde à numeração do Python, que começa em zero. Se fosse Matlab, que começa em um,  seria $k=j + (i-1)J$. Por outro lado,  $i$ e $j$ trocam se usarmos a habitual ordem de $i$ para linhas e $j$ para colunas! É só mais um exemplo de como é necessário estar atento às convenções!)

Por exemplo, usando o índice $k$, os pontos da grelha $(j,i,n)$ e $(j,i+1,n)$ tornam-se $(k,n)$ e $(k+J,n)$ respectivamente.
O nosso stencil fica nesta notação:  

$
\begin{eqnarray*}
U^{n+1}_k - U^n_k = & \sigma_x \left( U^n_{k+1} -2U^n_{k} + U^n_{k-1} + U^{n+1}_{k+1} -2U^{n+1}_{k} +U^{n+1}_{k-1}\right) +\\
      & \sigma_y \left( U^n_{k+J} -2U^n_{k} + U^n_{k-J} + U^{n+1}_{k+J} -2U^{n+1}_{k} +U^{n+1}_{k-J}\right) + \Delta t f(U^n_{k}).
\end{eqnarray*}
$

Reordenando esta expressão, temos:  

$
\begin{eqnarray*}
	-\sigma_y U^{n+1}_{k-J} -\sigma_x U^{n+1}_{k-1} & + (1 +2\sigma_x +2\sigma_y) U^{n+1}_{k} -\sigma_x U^{n+1}_{k+1} -\sigma_y U^{n+1}_{k+J} = \\
	&-\sigma_y U^{n}_{k-J} -\sigma_x U^{n}_{k-1} + (1 -2\sigma_x -2\sigma_y) U^{n}_{k} -\sigma_x U^{n}_{k+1} -\sigma_y U^{n}_{k+J} \Delta t f(U^n_{k}).
\end{eqnarray*}
$

Se escrevêssemos esta expressão em notação de matrizes víamos que temos matrizes por bandas tanto do lado esquerdo como do direito do sinal de "=". Do lado esquerdo a matriz (chamemos-lhe $A$) tem um "núcleo"  tridiagonal (tal como no caso CN em 1D) mas agora tem também elementos não-nulos $(-\sigma_y)$ na J-ésimas sub- e super-diagonais. Pelo seu lado a matriz do lado direito (chamemos-lhe $B$) é semelhante a $A$  mas com $(+\sigma_y)$ na J-ésimas sub- e super-diagonais. Inversão numérica de matrizes por bandas (como se chama às matrizes como $A$ e $B$) é dispendiosa por métodos standard (certamente não poderíamos usar o algoritmo de Thomas para matrizes tridiagonais, como no CN!). 

## O método de  *Alternating Direction Implicit (ADI)*
O algoritmo ADI usa um truque para o problema poder ser resolvido precisamente usando inversão de matrizes tridiagonais. Além disso permite facilmente uma implementação em paralelo. 

O truque consiste em dividir o passo no tempo $\Delta t$ em dois e aplicar um stencil diferente a cada um dos meios passos. Isto é possível porque o operador laplaciano pode ser visto como a soma de dois operadores, cada um dependente apenas de uma coordenada. Assim , para avançar um passo $\Delta t$ (de $n$ para $n+1$) no ponto da grelha $(j,i,n)$, primeiro calculamos $U^{n+1/2}_{j,i}$ e depois  calculamos $U^{n+1}_{j,i}$. Em ambos os passos o stencil é escolhido de modo a levar a matrizes tridiagonais.
Os dois stenceis ADI avançam as direções $x$ e $y$ na direcção do ponto temporal seguinte (stencil de diferenças finitas implícito vs explícito) alternadamente -- daí o nome do método.

$
\begin{eqnarray*}
	\frac{{\color{red}{ U^{n+1/2}_{j,i}}} - {\color{blue}{U^n_{j,i}}}}{\Delta t/2} =& {\color{red}{\frac{D}{\Delta x^2}\left( U^{n+1/2}_{j+1,i} -2U^{n+1/2}_{j,i} +U^{n+1/2}_{j-1,i}\right) }}\\
	+& {\color{blue}{\frac{D}{\Delta y^2}\left( U^n_{j,i+1} -2U^n_{j,i} + U^n_{j,i-1} \right) + f(U^n_{j,i})}} \\
	\frac{{\color{red}{U^{n+1}_{j,i}}} - {\color{blue}{U^{n+1/2}_{j,i}}}}{\Delta t/2} =& {\color{blue}{\frac{D}{\Delta x^2}\left( U^{n+1/2}_{j+1,i} -2U^{n+1/2}_{j,i} + U^{n+1/2}_{j-1,i} \right)}}\\
	+& {\color{red}{\frac{D}{\Delta y^2}\left( U^{n+1}_{j,i+1} -2U^{n+1}_{j,i} +U^{n+1}_{j,i-1}\right) + f(U^n_{j,i})}}
\end{eqnarray*}
$

Como se vê das expressões anteriores, parte (vermelho) corresponde a um cálculo implícito, e a restante parte (azul) corresponde a um cálculo explícito.

Sejam agora $\alpha_x \equiv \frac{D\Delta t}{\Delta x^2}$ e $\alpha_y \equiv \frac{D\Delta t}{\Delta y^2}$. Note-se que $2\sigma_x = \alpha_x$ e $2\sigma_y = \alpha_y$. Reordenando os stenceis, obtemos os seguintes sistemas de equações lineares:

$
-\alpha_x U^{n+1/2}_{j-1,i} +(1+2\alpha_x) U^{n+1/2}_{j,i} - \alpha_xU^{n+1/2}_{j+1,i} = \alpha_yU^n_{j,i-1} + (1-2\alpha_y)U^n_{j,i} + \alpha_y U^n_{j,i+1}+\Delta t f(U^n_{j,i})$

e

$-\alpha_y U^{n+1}_{j,i+1}+(1+2\alpha_y)U^{n+1}_{j,i} -\alpha_yU^{n+1}_{j,i-1} =  \alpha_x U^{n+1/2}_{j-1,i}+(1-2\alpha_x)U^{n+1/2}_{j,i}+\alpha_x U^{n+1/2}_{j+1,i}+ \Delta tf(U^{n+1/2}_{j	,i})
$

### Família de sistemas lineares na direcção x

Consideremos o stencil da Fig.(1), e vamos considerar a matriz definida por ($U^n_{j,i}$)
($j=0,\ldots,J-1, i=0,\ldots,I-1$, $n$ mantido constante) como um plano de concentração.

Note-se que para $i$ fixo (na nossa notação significa $y$ fixo) a primeira das duas equações anteriores define um sistema de equações lineares
semelhante ao sistema linear que obtivemos para o CN a 1D. Tal como nesse caso, teremos que alterar algumas entradas do sistema
para acomodar as condições fronteira de Neumann nas extremidades da grelha. Na verdade isso precisa ser feito em ambas as direcções $x$ e $y$.

A nossa primeira equação não precisa de alterações para aqueles pontos da grelha a que chamamos nodos livres, i.é,
para $j=1,\ldots,J-2, i=1,\ldots,I-2$, e $n=0,1,2,\ldots$  

$$
-\alpha_x U^{n+1/2}_{j-1,i} +(1+2\alpha_x) U^{n+1/2}_{j,i} - \alpha_xU^{n+1/2}_{j+1,i} = \alpha_y U^n_{j,i-1} + (1-2\alpha_y)U^n_{j,i} + \alpha_y U^n_{j,i+1}+\Delta t f(U^n_{j,i})
$$

Mas vejamos o que acontece nos pontos da grelha nas extremidades direita e esquerda, que têm $j=0$ ou $j=J-1$,respectivamente, e  $i=1,\ldots,I-2$. Tendo em conta que com CF de Neumann temos $U^{n+1/2}_{-1,i}=U^{n+1/2}_{0,i}$ e $U^{n+1/2}_{J,i} = U^{n+1/2}_{J-1,i}$ (se escolhermos discretizar a derivada com diferenças para a frente, claro!), vem:  

$$  (j=0): \quad (1+\alpha_x) U^{n+1/2}_{0,i} - \alpha_xU^{n+1/2}_{1,i} = \alpha_yU^n_{j,i-1} + (1-2\alpha_y)U^n_{0,i} + \alpha_y U^n_{0,i+1}+\Delta t f(U^n_{0,i})
$$

$$  (j=J-1): \quad -\alpha_x U^{n+1/2}_{J-2,i} +(1+\alpha_x) U^{n+1/2}_{J-1,i}  = \alpha_yU^n_{J-1,i-1} + (1-2\alpha_y)U^n_{J-1,i} + \alpha_y U^n_{J-1,i+1}+\Delta t f(U^n_{J-1,i})
$$

Do lado esquerdo das equações definidas por estas três expressões (variando $j=0,1, \ldots, J-1$) podemos discernir uma matriz da forma da que construímos para o CN a uma dimensão espacial. A matriz é a mesma para qualquer valor do índice $i=0,1,\ldots,I-1$.
Temos então as CF na direcção $x$ incorporadas. Antes de passar a fazer o mesmo na direcção $y$, relembramos só a nossa escolha de orientações/ordenamento. O nosso $x$ aumenta da esquerda para a direita e $y$ aumenta de baixo para cima. A nossa grelha tem a mesma orientação, pelo que o índice $j$ aumenta da esquerda para a direita e o índice $i$  aumenta de baixo para cima. Podemos dizer que temos uma orientação directa.

Vejamos o aspecto destas equações (CF incluídas) para $j=1,\ldots,J-2$ nos casos $i=0$ (fundo) e $i=I-1$ (topo) (onde é necessário aplicar também as CF!):  

$$
(i=0):\quad  -\alpha_x U^{n+1/2}_{j-1,0} +(1+2\alpha_x) U^{n+1/2}_{j,0} - \alpha_xU^{n+1/2}_{j+1,0} =  (1-\alpha_y)U^n_{j,0} + \alpha_y U^n_{j,1}+\Delta t f(U^n_{j,i})
$$

$$
(i=I-1): \quad  -\alpha_x U^{n+1/2}_{j-1,I-1} +(1+2\alpha_x) U^{n+1/2}_{j,I-1} - \alpha_xU^{n+1/2}_{j+1,I-1} = \alpha_y U^n_{j,I-2} + (1-\alpha_y)U^n_{j,I-1} +\Delta t f(U^n_{j,i})
$$

Para escrever estas equações de uma forma compacta, vamos definir uma fatia horizontal (da esquerda para a direita) como o vector:  

$$
\mathbf{U}^n_{x,i}  \equiv \left[ U^n_{0,i},U^n_{1,i},\ldots,U^n_{J-1,i}\right].
$$

Agora podemos combinar todas as equações anteriores e escrever a primeira das nossas duas famílias de sistemas lineares ADI de uma forma compacta:  

$$
A\mathbf{U}^{n+1/2}_{x,i}=\mathbf{b}_i+\mathbf{f}(\Delta t\mathbf{U}^{n}_{x,i}), i=0,\ldots,I-1,
$$

onde

$$  A = \begin{bmatrix}
	1+\alpha_x & -\alpha_x & 0 & 0 & 0 & \ldots & 0 & 0 & 0 & 0  \\
	-\alpha_x  & 1+\alpha_x & -\alpha_x & 0 &  0 & \ldots & 0 & 0 & 0 & 0  \\
	0  & -\alpha_x  & 1+\alpha_x & -\alpha_x & 0 &  \ldots & 0 & 0 & 0 & 0  \\
	0  & 0  & 0 & \ddots & \ddots &  \ddots & 0 & 0 & 0 & 0  \\
	0  & 0 & 0 & 0 & 0 &  \ldots & 0 & -\alpha_x  & 1+\alpha_x & -\alpha_x  \\
	0  & 0  & 0 & 0 & 0 &  \ldots & 0 & 0 & -\alpha_x  & 1+\alpha_x
\end{bmatrix}  $$

A forma do vector $\mathbf{b}_i$ depende do valor de $i$:

$$
i=I-1:\quad  \mathbf{b}_{I-1}= \begin{bmatrix}
\alpha_y U^n_{0,I-2} + (1-\alpha_y)U^n_{0,I-1} \\
\vdots\\
\alpha_y U^n_{J-1,I-2} + (1-\alpha_y)U^n_{J-1,I-1} 
\end{bmatrix}
$$

$$
i=I-2,\ldots,1:\quad  \mathbf{b}_i=\begin{bmatrix}
\alpha_y U^n_{0,i-1} + (1-2\alpha_y)U^n_{0,i} + \alpha_y U^n_{0,i+1}\\
\vdots \\
\alpha_y U^n_{J-1,i-1} + (1-2\alpha_y)U^n_{J-1,i} + \alpha_y U^n_{J-1,i+1}
\end{bmatrix}
$$

$$
i=0: \quad  \mathbf{b}_0=\begin{bmatrix}
(1-\alpha_y)U^n_{0,0} +\alpha_y U^n_{0,1}\\
\vdots \\
(1-\alpha_y)U^n_{J-1,0} +\alpha_y U^n_{J-1,1}
\end{bmatrix}
$$

O vector do termo de reacção é:

$$  \mathbf{f}(\mathbf{U}^{n}_{x,i})= \Delta t . f(U^n_{j,I-1},U^n_{j,I-2},\ldots, U^n_{j,0}) $$

### Família de sistemas lineares na direcção y

Comecemos por definir uma fatia vertical no nosso plano (de cima para baixo) como:

$$ \mathbf{U}_{y,j}^n = \begin{bmatrix}U_{j,I-1}^n, & U_{j,I-2}^n, & \ldots, &
U_{j,0}^n \end{bmatrix}  $$


Seguindo um procedimento equivalente ao de cima para a segunda família de equações ADI, obtemos:  

$$ C\mathbf{U}^{n+1}_{y,j}=\mathbf{d}_j+\mathbf{f}(\Delta t\mathbf{U}^{n+1/2}_{y,j}), j=0,\ldots,J-1, $$
onde  

$$  C =\begin{bmatrix}
1+\alpha_y & -\alpha_y & 0 & 0 & 0 & \ldots & 0 & 0 & 0 & 0  \\
-\alpha_y  & 1+2\alpha_y & -\alpha_y & 0 &  0 & \ldots & 0 & 0 & 0 & 0  \\
0  & -\alpha_y  & 1+2\alpha_y & -\alpha_y & 0 &  \ldots & 0 & 0 & 0 & 0  \\
0  & 0  & 0 & \ddots & \ddots &  \ddots & 0 & 0 & 0 & 0  \\
0  & 0 & 0 & 0 & 0 &  \ldots & 0 & -\alpha_y  & 1+2\alpha_y & -\alpha_y  \\
0  & 0  & 0 & 0 & 0 &  \ldots & 0 & 0 & -\alpha_y  & 1+\alpha_y
\end{bmatrix},
$$

A forma do vector $\mathbf{d}_j$ depende do valor de $j$:  

$$ 
j=0: \quad  \mathbf{d}_0=\begin{bmatrix}
 (1-\alpha_x)U^{n+1/2}_{0,I-1} + \alpha_x U^{n+1/2}_{1,I-1}\\
 (1-\alpha_x)U^{n+1/2}_{0,I-2} + \alpha_x U^{n+1/2}_{1,I-2}\\
 \vdots \\
 (1-\alpha_x)U^{n+1/2}_{0,0} + \alpha_x U^{n+1/2}_{1,0}
 \end{bmatrix}
 $$
 
 $$
 j = 1,\ldots,J-2:\quad  \mathbf{d}_j=\begin{bmatrix}
 \alpha_x U^{n+1/2}_{j-1,I-1} + (1-2\alpha_x)U^{n+1/2}_{j,I-1} + \alpha_x U^{n+1/2}_{j+1,I-1}\\
  \alpha_x U^{n+1/2}_{j-1,I-2} + (1-2\alpha_x)U^{n+1/2}_{j,I-2} + \alpha_x U^{n+1/2}_{j+1,I-2} \\
 \vdots \\
 \alpha_x U^{n+1/2}_{j-1,0} + (1-2\alpha_x)U^{n+1/2}_{j,0} + \alpha_x U^{n+1/2}_{j+1,0}
 \end{bmatrix}
 $$



$$
j=J-1: \quad  \mathbf{d}_{J-1}=\begin{bmatrix}
\alpha_x U^{n+1/2}_{J-2,I-1} + (1-\alpha_x)U^{n+1/2}_{J-1,I-1} \\
\alpha_x U^{n+1/2}_{J-2,I-2} + (1-\alpha_x)U^{n+1/2}_{J-1,I-2} \\
\vdots \\
\alpha_x U^{n+1/2}_{J-2,0} + (1-\alpha_x)U^{n+1/2}_{J-1,0} 
\end{bmatrix}
$$

O vector do termo de reacção é:

$$  \mathbf{f}(\mathbf{U}^{n}_{y,j})= \Delta t .f(U^{n+1/2}_{j,I-1},U^{n+1/2}_{j,I-2},\ldots, U^{n+1/2}_{j,0}) $$

### Paralelismo no ADI

Em resumo, o stencil ADI gera duas famílias de sistemas lineares que precisamos resolver alternadamente:  

$$ A\mathbf{U}^{n+1/2}_{x,i}=\mathbf{b}_i+\mathbf{f}(\Delta t\mathbf{U}^{n}_{x,i}),\; i=0,\ldots,I-1  $$

$$ C\mathbf{U}^{n+1}_{y,j}=\mathbf{d}_j+\mathbf{f}(\Delta t\mathbf{U}^{n+1/2}_{y,j}),\; j=0,\ldots,J-1,  $$

Note-se que podemos resolver os sistemas da primeira família em paralelo. Para ver tal, basta considerar quaisquer dois sistemas dessa família (distintos $i$'s, $i_1$ e $i_2$), e notar que cada um depende apenas dos $\mathbf{U}^{n}_{x,i} $ do seu próprio $i$.

O mesmo raciocínio aplicado à segunda família de sistemas lineares permite ver que cada índice $j$ corresponde a um sistema independente dos demais. Como tal podem ser resolvidos em paralelo. (Ver, p.ex.,  https://en.wikipedia.org/wiki/Kronecker_sum_of_discrete_Laplacians.)

## Bibliografia
[1] Mehdi Dehghan, _Second-order schemes for a boundary value problem with Neumann's boundary conditions_, Journal of Computational and Applied Mathematics, *138*, no 1, 173 - 184 (2002)

[2] S.P Venkateshan e P. Swaminathan, _Computational Methods in Engineering_, Academic Press (2014)(cap. 14, e rever cap.13)
